# Discharge Notes and Readmission Rates: EDA

This notebook is for the initial exploratory data analysis of the dataset.

## Overview

## Business Understanding

## Data Understanding

## Import

Import all the relevant libraries

In [1]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt

In [2]:
#set the pandas display column width as max to read in all the text
#pd.set_option('display.max_colwidth', None)

## Functions

In [3]:
def group_categories(df, category):
    group = df.groupby(['SUBJECT_ID', 'HADM_ID'])
    result = group.agg({category:lambda x:list(set(x.tolist()))})
    result.reset_index(inplace=True)
    result[category] = [', '.join(map(str, l)) for l in result[category]]
    return result

## Exploratory Data Analysis (EDA)

INSERT DESCRIPTION

### Admission Data

In [4]:
admin_df = pd.read_csv('data/ADMISSIONS.csv.gz', compression='gzip')

In [5]:
admin_df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 58976 entries, 0 to 58975
Data columns (total 19 columns):
 #   Column                Non-Null Count  Dtype 
---  ------                --------------  ----- 
 0   ROW_ID                58976 non-null  int64 
 1   SUBJECT_ID            58976 non-null  int64 
 2   HADM_ID               58976 non-null  int64 
 3   ADMITTIME             58976 non-null  object
 4   DISCHTIME             58976 non-null  object
 5   DEATHTIME             5854 non-null   object
 6   ADMISSION_TYPE        58976 non-null  object
 7   ADMISSION_LOCATION    58976 non-null  object
 8   DISCHARGE_LOCATION    58976 non-null  object
 9   INSURANCE             58976 non-null  object
 10  LANGUAGE              33644 non-null  object
 11  RELIGION              58518 non-null  object
 12  MARITAL_STATUS        48848 non-null  object
 13  ETHNICITY             58976 non-null  object
 14  EDREGTIME             30877 non-null  object
 15  EDOUTTIME             30877 non-null

In [6]:
admin_df.head()

,ROW_ID,SUBJECT_ID,HADM_ID,ADMITTIME,DISCHTIME,DEATHTIME,ADMISSION_TYPE,ADMISSION_LOCATION,DISCHARGE_LOCATION,INSURANCE,LANGUAGE,RELIGION,MARITAL_STATUS,ETHNICITY,EDREGTIME,EDOUTTIME,DIAGNOSIS,HOSPITAL_EXPIRE_FLAG,HAS_CHARTEVENTS_DATA
0,21,22,165315,2196-04-09 12:26:00,2196-04-10 15:54:00,NaN,EMERGENCY,EMERGENCY ROOM ADMIT,DISC-TRAN CANCER/CHLDRN H,Private,NaN,UNOBTAINABLE,MARRIED,WHITE,2196-04-09 10:06:00,2196-04-09 13:24:00,BENZODIAZEPINE OVERDOSE,0,1
1,22,23,152223,2153-09-03 07:15:00,2153-09-08 19:10:00,NaN,ELECTIVE,PHYS REFERRAL/NORMAL DELI,HOME HEALTH CARE,Medicare,NaN,CATHOLIC,MARRIED,WHITE,NaN,NaN,CORONARY ARTERY DISEASE\CORONARY ARTERY BYPASS...,0,1
2,23,23,124321,2157-10-18 19:34:00,2157-10-25 14:00:00,NaN,EMERGENCY,TRANSFER FROM HOSP/EXTRAM,HOME HEALTH CARE,Medicare,ENGL,CATHOLIC,MARRIED,WHITE,NaN,NaN,BRAIN MASS,0,1
3,24,24,161859,2139-06-06 16:14:00,2139-06-09 12:48:00,NaN,EMERGENCY,TRANSFER FROM HOSP/EXTRAM,HOME,Private,NaN,PROTESTANT QUAKER,SINGLE,WHITE,NaN,NaN,INTERIOR MYOCARDIAL INFARCTION,0,1
4,25,25,129635,2160-11-02 02:06:00,2160-11-05 14:55:00,NaN,EMERGENCY,EMERGENCY ROOM ADMIT,HOME,Private,NaN,UNOBTAINABLE,MARRIED,WHITE,2160-11-02 01:01:00,2160-11-02 04:27:00,ACUTE CORONARY SYNDROME,0,1


In [7]:
admin_df['ADMISSION_TYPE'].value_counts()

EMERGENCY    42071
NEWBORN       7863
ELECTIVE      7706
URGENT        1336
Name: ADMISSION_TYPE, dtype: int64

In [8]:
columns_to_drop = ['DISCHTIME','DEATHTIME','EDREGTIME','EDOUTTIME','HOSPITAL_EXPIRE_FLAG']
admin_df.drop(columns_to_drop, axis=1, inplace=True)

### Chart Notes

In [9]:
notes_df = pd.read_csv('data/NOTEEVENTS.csv.gz', compression='gzip')

/Users/arthur/opt/anaconda3/envs/main_env/lib/python3.8/site-packages/IPython/core/interactiveshell.py:3417: DtypeWarning: Columns (4,5) have mixed types.Specify dtype option on import or set low_memory=False.
  exec(code_obj, self.user_global_ns, self.user_ns)


In [10]:
notes_df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 2083180 entries, 0 to 2083179
Data columns (total 11 columns):
 #   Column       Dtype  
---  ------       -----  
 0   ROW_ID       int64  
 1   SUBJECT_ID   int64  
 2   HADM_ID      float64
 3   CHARTDATE    object 
 4   CHARTTIME    object 
 5   STORETIME    object 
 6   CATEGORY     object 
 7   DESCRIPTION  object 
 8   CGID         float64
 9   ISERROR      float64
 10  TEXT         object 
dtypes: float64(3), int64(2), object(6)
memory usage: 174.8+ MB


In [11]:
notes_df.head()

,ROW_ID,SUBJECT_ID,HADM_ID,CHARTDATE,CHARTTIME,STORETIME,CATEGORY,DESCRIPTION,CGID,ISERROR,TEXT
0,174,22532,167853.0,2151-08-04,NaN,NaN,Discharge summary,Report,NaN,NaN,Admission Date: [**2151-7-16**] Dischar...
1,175,13702,107527.0,2118-06-14,NaN,NaN,Discharge summary,Report,NaN,NaN,Admission Date: [**2118-6-2**] Discharg...
2,176,13702,167118.0,2119-05-25,NaN,NaN,Discharge summary,Report,NaN,NaN,Admission Date: [**2119-5-4**] D...
3,177,13702,196489.0,2124-08-18,NaN,NaN,Discharge summary,Report,NaN,NaN,Admission Date: [**2124-7-21**] ...
4,178,26880,135453.0,2162-03-25,NaN,NaN,Discharge summary,Report,NaN,NaN,Admission Date: [**2162-3-3**] D...


In [12]:
notes_df = notes_df[notes_df['ISERROR'] != 1]
notes_df = notes_df[notes_df['CATEGORY'] == 'Discharge summary']

In [13]:
columns_to_drop = ['ROW_ID','CHARTDATE','CHARTTIME','STORETIME','CATEGORY','DESCRIPTION','CGID','ISERROR']
notes_df.drop(columns_to_drop, axis=1, inplace=True)

In [14]:
notes_df.head()

,SUBJECT_ID,HADM_ID,TEXT
0,22532,167853.0,Admission Date: [**2151-7-16**] Dischar...
1,13702,107527.0,Admission Date: [**2118-6-2**] Discharg...
2,13702,167118.0,Admission Date: [**2119-5-4**] D...
3,13702,196489.0,Admission Date: [**2124-7-21**] ...
4,26880,135453.0,Admission Date: [**2162-3-3**] D...


### CPT Codes

In [ ]:
cpt_events_df = pd.read_csv('data/CPTEVENTS.csv.gz', compression='gzip')

In [ ]:
cpt_events_df.info()

In [17]:
cpt_events_df.head()

,ROW_ID,SUBJECT_ID,HADM_ID,COSTCENTER,CHARTDATE,CPT_CD,CPT_NUMBER,CPT_SUFFIX,TICKET_ID_SEQ,SECTIONHEADER,SUBSECTIONHEADER,DESCRIPTION
0,317,11743,129545,ICU,NaN,99232,99232.0,NaN,6.0,Evaluation and management,Hospital inpatient services,NaN
1,318,11743,129545,ICU,NaN,99232,99232.0,NaN,7.0,Evaluation and management,Hospital inpatient services,NaN
2,319,11743,129545,ICU,NaN,99232,99232.0,NaN,8.0,Evaluation and management,Hospital inpatient services,NaN
3,320,11743,129545,ICU,NaN,99232,99232.0,NaN,9.0,Evaluation and management,Hospital inpatient services,NaN
4,321,6185,183725,ICU,NaN,99223,99223.0,NaN,1.0,Evaluation and management,Hospital inpatient services,NaN


In [18]:
cpt_events_df = cpt_events_df[['SUBJECT_ID', 'HADM_ID', 'CPT_CD']]

### Diagnosis ICD Codes

In [19]:
diagnoses_icd_df = pd.read_csv('data/DIAGNOSES_ICD.csv.gz', compression='gzip')

In [20]:
diagnoses_icd_df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 651047 entries, 0 to 651046
Data columns (total 5 columns):
 #   Column      Non-Null Count   Dtype  
---  ------      --------------   -----  
 0   ROW_ID      651047 non-null  int64  
 1   SUBJECT_ID  651047 non-null  int64  
 2   HADM_ID     651047 non-null  int64  
 3   SEQ_NUM     651000 non-null  float64
 4   ICD9_CODE   651000 non-null  object 
dtypes: float64(1), int64(3), object(1)
memory usage: 24.8+ MB


In [21]:
diagnoses_icd_df.head()

,ROW_ID,SUBJECT_ID,HADM_ID,SEQ_NUM,ICD9_CODE
0,1297,109,172335,1.0,40301
1,1298,109,172335,2.0,486
2,1299,109,172335,3.0,58281
3,1300,109,172335,4.0,5855
4,1301,109,172335,5.0,4254


In [22]:
diagnoses_icd_df = diagnoses_icd_df[['SUBJECT_ID', 'HADM_ID', 'ICD9_CODE']]

### DRG Codes

In [23]:
drg_codes_df = pd.read_csv('data/DRGCODES.csv.gz', compression='gzip')

In [24]:
drg_codes_df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 125557 entries, 0 to 125556
Data columns (total 8 columns):
 #   Column         Non-Null Count   Dtype  
---  ------         --------------   -----  
 0   ROW_ID         125557 non-null  int64  
 1   SUBJECT_ID     125557 non-null  int64  
 2   HADM_ID        125557 non-null  int64  
 3   DRG_TYPE       125557 non-null  object 
 4   DRG_CODE       125557 non-null  int64  
 5   DESCRIPTION    125494 non-null  object 
 6   DRG_SEVERITY   66634 non-null   float64
 7   DRG_MORTALITY  66634 non-null   float64
dtypes: float64(2), int64(4), object(2)
memory usage: 7.7+ MB


In [25]:
drg_codes_df.head()

,ROW_ID,SUBJECT_ID,HADM_ID,DRG_TYPE,DRG_CODE,DESCRIPTION,DRG_SEVERITY,DRG_MORTALITY
0,342,2491,144486,HCFA,28,"TRAUMATIC STUPOR & COMA, COMA <1 HR AGE >17 WI...",NaN,NaN
1,343,24958,162910,HCFA,110,MAJOR CARDIOVASCULAR PROCEDURES WITH COMPLICAT...,NaN,NaN
2,344,18325,153751,HCFA,390,NEONATE WITH OTHER SIGNIFICANT PROBLEMS,NaN,NaN
3,345,17887,182692,HCFA,14,SPECIFIC CEREBROVASCULAR DISORDERS EXCEPT TRAN...,NaN,NaN
4,346,11113,157980,HCFA,390,NEONATE WITH OTHER SIGNIFICANT PROBLEMS,NaN,NaN


In [26]:
drg_codes_df = drg_codes_df[['SUBJECT_ID','HADM_ID','DRG_CODE']]

### Procedure ICD Codes

In [27]:
proced_icd_df = pd.read_csv('data/PROCEDURES_ICD.csv.gz', compression='gzip')

In [28]:
proced_icd_df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 240095 entries, 0 to 240094
Data columns (total 5 columns):
 #   Column      Non-Null Count   Dtype
---  ------      --------------   -----
 0   ROW_ID      240095 non-null  int64
 1   SUBJECT_ID  240095 non-null  int64
 2   HADM_ID     240095 non-null  int64
 3   SEQ_NUM     240095 non-null  int64
 4   ICD9_CODE   240095 non-null  int64
dtypes: int64(5)
memory usage: 9.2 MB


In [29]:
proced_icd_df.head()

,ROW_ID,SUBJECT_ID,HADM_ID,SEQ_NUM,ICD9_CODE
0,944,62641,154460,3,3404
1,945,2592,130856,1,9671
2,946,2592,130856,2,3893
3,947,55357,119355,1,9672
4,948,55357,119355,2,331


In [30]:
proced_icd_df = proced_icd_df[['SUBJECT_ID','HADM_ID','ICD9_CODE']]

## Removing Duplicate Admissions

Since some patients are readmitted multiple times at the hospital, I wanted to remove the last visit they had to the hospital since I cannot use the discharge note for the last visit since there would be no subsequent readmissions

In [31]:
#sort the dates in ascending order, so that if we were to delete the last duplicate, it would be the latest admissions
admin_df.sort_values('ADMITTIME', inplace=True)

In [32]:
#create a dataframe on the number of times a patient was admitted
dupli_admin_df = pd.DataFrame(admin_df.groupby('SUBJECT_ID')['HADM_ID'].count())
dupli_admin_df.rename(columns={'HADM_ID': 'ADMIT_COUNTS'}, inplace=True)

In [33]:
#printout how many patients were admitted once vs admitted multiple times
print(dupli_admin_df[dupli_admin_df['ADMIT_COUNTS']==1].count())
print(dupli_admin_df[dupli_admin_df['ADMIT_COUNTS']>1].count())

ADMIT_COUNTS    38983
dtype: int64
ADMIT_COUNTS    7537
dtype: int64


We can see here that there is clearly a class imbalance that we will have to take into account when modeling.

In [34]:
#merged the admission counts back into the main dataframe
new_df = admin_df.merge(dupli_admin_df,on='SUBJECT_ID')

#separate out the people who were never readmitted
mod_df = new_df[new_df['ADMIT_COUNTS']==1]
mod_df['ADMISSION_TYPE'].value_counts()

EMERGENCY    25417
NEWBORN       7603
ELECTIVE      5066
URGENT         897
Name: ADMISSION_TYPE, dtype: int64

In [35]:
#separate out the people who were readmitted multiple times
duplicate_df = new_df[new_df['ADMIT_COUNTS']>1]

#create a copy of the dataframe with the duplicates
df1 = duplicate_df.copy()

In [36]:
#create a list of the unique number of times people were readmitted
admin_counts = sorted(df1['ADMIT_COUNTS'].unique())

In [37]:
#for every admission counts
for i in admin_counts:
    #find the patients associated with that admission count
    df2 = df1[df1['ADMIT_COUNTS']==i]
    
    #this while loop is to iterate the dataframe and retrieving the first duplicate of the main dataframe
    #and then the main dataframe becomes the deleted out duplicates, this process is repeated until the 
    #admin counts reaches 1 at 'i -=1'
    while i > 1:
        #this dataframe holds the duplicates
        df3 = df2.loc[df2.duplicated(subset='SUBJECT_ID')]
        #this datafame holds the de-duped dataframe
        df4 = df2.drop_duplicates('SUBJECT_ID')
        #add the deduped dataframe to the main dataframe
        mod_df = pd.concat([mod_df, df4])
        df2 = df3
        i -= 1

In [38]:
#review the modified DataFrame to see if it worked
mod_df[mod_df['ADMIT_COUNTS']==3].sort_values('SUBJECT_ID').head(20)

,ROW_ID,SUBJECT_ID,HADM_ID,ADMITTIME,ADMISSION_TYPE,ADMISSION_LOCATION,DISCHARGE_LOCATION,INSURANCE,LANGUAGE,RELIGION,MARITAL_STATUS,ETHNICITY,DIAGNOSIS,HAS_CHARTEVENTS_DATA,ADMIT_COUNTS
18056,37,36,122659,2131-05-12 19:49:00,EMERGENCY,EMERGENCY ROOM ADMIT,REHAB/DISTINCT PART HOSP,Medicare,ENGL,NOT SPECIFIED,MARRIED,WHITE,CHEST PAIN/SHORTNESS OF BREATH,1,3
18055,36,36,182104,2131-04-30 07:15:00,EMERGENCY,CLINIC REFERRAL/PREMATURE,HOME HEALTH CARE,Medicare,ENGL,NOT SPECIFIED,MARRIED,WHITE,CORONARY ARTERY DISEASE\CORONARY ARTERY BYPASS...,1,3
8491,112,107,191941,2115-02-20 17:41:00,EMERGENCY,CLINIC REFERRAL/PREMATURE,HOME,Private,ENGL,CATHOLIC,MARRIED,HISPANIC OR LATINO,AV FISTULA;ANGIOPLASTY,1,3
8492,113,107,182383,2121-11-30 19:24:00,EMERGENCY,CLINIC REFERRAL/PREMATURE,HOME HEALTH CARE,Medicare,ENGL,CATHOLIC,SEPARATED,HISPANIC OR LATINO,HYPERKALEMIA;CONGESTIVE HEART FAILURE,1,3
18407,186,138,188284,2133-12-21 23:52:00,EMERGENCY,TRANSFER FROM HOSP/EXTRAM,HOME,Private,ENGL,CATHOLIC,MARRIED,WHITE,HEMATURIA;RENAL MASS,1,3
18406,185,138,108120,2131-10-31 08:00:00,ELECTIVE,PHYS REFERRAL/NORMAL DELI,HOME,Private,NaN,CATHOLIC,MARRIED,WHITE,AORTIC STENOSIS\ AORTIC VALVE REPLACEMENT/SDA,1,3
10164,205,154,162891,2118-04-05 18:11:00,URGENT,PHYS REFERRAL/NORMAL DELI,HOME HEALTH CARE,Private,NaN,EPISCOPALIAN,MARRIED,WHITE,UNSTABLE ANGINA\CATH WITH BRACHY S/B,1,3
10163,204,154,111735,2117-12-29 21:36:00,EMERGENCY,EMERGENCY ROOM ADMIT,HOME,Private,NaN,EPISCOPALIAN,MARRIED,WHITE,ACUTE MYOCARDIAL INFARCTION,1,3
20287,308,236,182562,2135-05-26 11:28:00,EMERGENCY,CLINIC REFERRAL/PREMATURE,HOME,Private,ENGL,CATHOLIC,MARRIED,PATIENT DECLINED TO ANSWER,POST OP BLEEDING,1,3
20286,307,236,151459,2134-10-04 01:19:00,EMERGENCY,PHYS REFERRAL/NORMAL DELI,HOME HEALTH CARE,Private,ENGL,CATHOLIC,MARRIED,PATIENT DECLINED TO ANSWER,END STAGE LIVER DISEASE,1,3


As we can see the patients who were admitted three times showed up twice in our dataframe, meaning that one duplicate has successfully been removed. Below, we pull subject ID 36 as an example from the original dataframe to see that the last admission visit was removed.

In [39]:
#double checking that the last admissions time was the one that was removed
admin_df[admin_df['SUBJECT_ID'] == 36]

,ROW_ID,SUBJECT_ID,HADM_ID,ADMITTIME,ADMISSION_TYPE,ADMISSION_LOCATION,DISCHARGE_LOCATION,INSURANCE,LANGUAGE,RELIGION,MARITAL_STATUS,ETHNICITY,DIAGNOSIS,HAS_CHARTEVENTS_DATA
15,36,36,182104,2131-04-30 07:15:00,EMERGENCY,CLINIC REFERRAL/PREMATURE,HOME HEALTH CARE,Medicare,ENGL,NOT SPECIFIED,MARRIED,WHITE,CORONARY ARTERY DISEASE\CORONARY ARTERY BYPASS...,1
16,37,36,122659,2131-05-12 19:49:00,EMERGENCY,EMERGENCY ROOM ADMIT,REHAB/DISTINCT PART HOSP,Medicare,ENGL,NOT SPECIFIED,MARRIED,WHITE,CHEST PAIN/SHORTNESS OF BREATH,1
17,38,36,165660,2134-05-10 11:30:00,ELECTIVE,PHYS REFERRAL/NORMAL DELI,LONG TERM CARE HOSPITAL,Medicare,ENGL,NOT SPECIFIED,MARRIED,WHITE,VENTRAL HERNIA/SDA,1


In [40]:
#convert all the admission counts to 1 (readmission occurred after this admissions) or 0 (no readmission)
mod_df.loc[mod_df['ADMIT_COUNTS'] == 1, 'ADMIT_COUNTS'] = 0
mod_df.loc[mod_df['ADMIT_COUNTS'] > 1, 'ADMIT_COUNTS'] = 1
mod_df.rename(columns={'ADMIT_COUNTS': 'READMISSION'}, inplace=True)

## Grouping Codes by Subject and Admission IDs

In [41]:
cpt_events_df = group_categories(cpt_events_df, 'CPT_CD')

In [42]:
diagnoses_icd_df = group_categories(diagnoses_icd_df, 'ICD9_CODE')
diagnoses_icd_df.rename(columns={'ICD9_CODE': 'DIAG_ICD9_CODE'}, inplace=True)

In [43]:
drg_codes_df = group_categories(drg_codes_df, 'DRG_CODE')

In [44]:
proced_icd_df = group_categories(proced_icd_df, 'ICD9_CODE')
proced_icd_df.rename(columns={'ICD9_CODE': 'PROCED_ICD9_CODE'}, inplace=True)

In [45]:
merged_df = cpt_events_df.merge(diagnoses_icd_df, on=['SUBJECT_ID', 'HADM_ID'], how='outer')
merged_df = merged_df.merge(drg_codes_df, on=['SUBJECT_ID', 'HADM_ID'], how='outer')
merged_df = merged_df.merge(proced_icd_df, on=['SUBJECT_ID', 'HADM_ID'], how='outer')
merged_df

,SUBJECT_ID,HADM_ID,CPT_CD,DIAG_ICD9_CODE,DRG_CODE,PROCED_ICD9_CODE
0,3,145834,"94003, 94002","41071, 78559, 5849, 0389, 6826, 4275, 4280, 42...",416,"9604, 8964, 966, 9672, 9962, 3893"
1,4,185777,"99232, 99233, 99238, 99223, 99231","5715, 7907, 7994, V090, E9317, 1363, 2763, 041...",489,"8872, 3323, 3893"
2,6,107064,"99232, 99233, 99239, 99255, 99291, 99231","2767, V1582, 40391, 2859, 4440, 9972, 2753, 2766",302,"5569, 9904, 3957, 91, 3806"
3,9,150750,94003,"431, 5849, 5070, 4019, 4280, 2765",14,"9672, 9604"
4,11,194540,"99239, 99233",1913,"2, 211","9925, 113, 9229, 159"
...,...,...,...,...,...,...
58971,99017,103056,NaN,"41071, 4439, 2859, 3659, 496, 41401","1742, 247","66, 40, 47, 3606, 3607"
58972,99088,165352,NaN,"72989, 43889, 4019, 2326","578, 3611","864, 8661"
58973,99487,188460,NaN,"25000, V5867, 2859, 53081, 2689, 41021, 4019, ...","1742, 247","66, 40, 3722, 45, 3607, 8856"
58974,99650,190854,NaN,"25050, 2724, 5363, 40391, 4139, 36201, 25060, ...","554, 1733","3929, 3995"


In [46]:
merged_df.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 58976 entries, 0 to 58975
Data columns (total 6 columns):
 #   Column            Non-Null Count  Dtype 
---  ------            --------------  ----- 
 0   SUBJECT_ID        58976 non-null  int64 
 1   HADM_ID           58976 non-null  int64 
 2   CPT_CD            44148 non-null  object
 3   DIAG_ICD9_CODE    58976 non-null  object
 4   DRG_CODE          58890 non-null  object
 5   PROCED_ICD9_CODE  52243 non-null  object
dtypes: int64(2), object(4)
memory usage: 3.1+ MB


In [50]:
mod2_df = mod_df.merge(merged_df, on=['SUBJECT_ID', 'HADM_ID'], how='outer')
mod2_df = mod2_df.merge(notes_df, on=['SUBJECT_ID', 'HADM_ID'], how='outer')
mod2_df

,ROW_ID,SUBJECT_ID,HADM_ID,ADMITTIME,ADMISSION_TYPE,ADMISSION_LOCATION,DISCHARGE_LOCATION,INSURANCE,LANGUAGE,RELIGION,MARITAL_STATUS,ETHNICITY,DIAGNOSIS,HAS_CHARTEVENTS_DATA,READMISSION,CPT_CD,DIAG_ICD9_CODE,DRG_CODE,PROCED_ICD9_CODE,TEXT
0,53486.0,82574,118464,2100-06-07 19:59:00,EMERGENCY,CLINIC REFERRAL/PREMATURE,HOME,Medicaid,ENGL,CATHOLIC,SINGLE,OTHER,HYPOTENSION,1.0,0.0,"99233, 99291, 99232","2449, 1991, 28411, 2720, V4589, 4400, 28529, 3...","2072, 2073, 314",9229,Admission Date: [**2100-6-7**] D...
1,35574.0,29156,161773,2100-06-09 01:39:00,EMERGENCY,EMERGENCY ROOM ADMIT,DEAD/EXPIRED,Private,CANT,UNOBTAINABLE,MARRIED,WHITE,RESPIRATORY DISTRESS,1.0,0.0,"99253, 99292, 99233, 94002, 94003, 99291, 99239","5990, 2948, 2639, 51881, 4821, 2859, 05312, 53...","1304, 565","9604, 966, 9672, 3893, 9915",Admission Date: [**2100-6-9**] D...
2,14753.0,12001,173927,2100-06-14 04:55:00,EMERGENCY,EMERGENCY ROOM ADMIT,SNF,Medicare,NaN,CATHOLIC,MARRIED,WHITE,RT BASAL GANGLIA BLEEDING,1.0,0.0,NaN,"7070, 2720, 725, 5990, 4019, 3310, 431, 29410",14,"4513, 966, 4311",Admission Date: [**2100-6-14**] Dischar...
3,14753.0,12001,173927,2100-06-14 04:55:00,EMERGENCY,EMERGENCY ROOM ADMIT,SNF,Medicare,NaN,CATHOLIC,MARRIED,WHITE,RT BASAL GANGLIA BLEEDING,1.0,0.0,NaN,"7070, 2720, 725, 5990, 4019, 3310, 431, 29410",14,"4513, 966, 4311",Admission Date: [**2100-6-14**] Dischar...
4,25723.0,21081,159656,2100-06-14 12:02:00,EMERGENCY,EMERGENCY ROOM ADMIT,HOME,Medicaid,NaN,NOT SPECIFIED,SINGLE,BLACK/AFRICAN AMERICAN,SHORTNESS OF BREATH,1.0,0.0,"99233, 99291","7863, 042, E8500, 51881, 30470, 96501, 9708, 4...",87,NaN,Admission Date: [**2100-6-14**] Discharge...
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
65897,NaN,98761,182540,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,"5370, 34690, 7101, 78909, 311, 56782, 42789, 5...","394, 2523",9702,NaN
65898,NaN,98761,184477,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,"5363, 5641","392, 2541",4311,NaN
65899,NaN,99088,165352,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,"72989, 4019, 43889, 2326","578, 3611","864, 8661",NaN
65900,NaN,99650,190854,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,"2724, 25040, 41401, 44022, 25050, 3572, 4139, ...","554, 1733","3929, 3995",Admission Date: [**2152-6-7**] Discharg...


In [51]:
mod2_df.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 65902 entries, 0 to 65901
Data columns (total 20 columns):
 #   Column                Non-Null Count  Dtype  
---  ------                --------------  -----  
 0   ROW_ID                56177 non-null  float64
 1   SUBJECT_ID            65902 non-null  int64  
 2   HADM_ID               65902 non-null  int64  
 3   ADMITTIME             56177 non-null  object 
 4   ADMISSION_TYPE        56177 non-null  object 
 5   ADMISSION_LOCATION    56177 non-null  object 
 6   DISCHARGE_LOCATION    56177 non-null  object 
 7   INSURANCE             56177 non-null  object 
 8   LANGUAGE              29980 non-null  object 
 9   RELIGION              55624 non-null  object 
 10  MARITAL_STATUS        45720 non-null  object 
 11  ETHNICITY             56177 non-null  object 
 12  DIAGNOSIS             56174 non-null  object 
 13  HAS_CHARTEVENTS_DATA  56177 non-null  float64
 14  READMISSION           56177 non-null  float64
 15  CPT_CD             